### Lag va rolling featurelar (USD/UZS)

Quyidagi kod `rate` bo'yicha 1, 7 va 30 kunlik lag va rolling xususiyatlarni hosil qiladi.

In [21]:
import pandas as pd
from pathlib import Path

In [33]:
# Faylni yuklaymiz
data_path = Path('datasets/usd_rates_filled.csv')
df = pd.read_csv(data_path)

# Ustun nomlarini qulay foydalanish uchun kichik harflarga o'girib olamiz
df.columns = [c.lower() for c in df.columns]

# Sanani datetime formatiga o'tkazib, vaqt bo'yicha tartiblaymiz
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

# Faqat kerakli ustunlarni olamiz: sana va kurs (rate)
df = df[['date', 'rate']].reset_index(drop=True)

df.head()

,date,rate
0,2018-10-26,"8,226.10"
1,2018-10-27,"8,226.10"
2,2018-10-28,"8,226.10"
3,2018-10-29,"8,226.10"
4,2018-10-30,"8,236.87"


In [34]:
# Lag va rolling oynalar (kunlarda)
lags = [1, 7, 30]
windows = [1, 7, 30]

features = df.copy()

for lag in lags:
    # "lag_{lag}" — kursning avvalgi {lag} kunlik qiymati
    features[f'lag_{lag}'] = features['rate'].shift(lag)

for win in windows:
    # "roll_min_{win}" — so'nggi {win} kun ichidagi minimal kurs
    features[f'roll_min_{win}'] = features['rate'].rolling(win).min()
    # "roll_max_{win}" — so'nggi {win} kun ichidagi maksimal kurs
    features[f'roll_max_{win}'] = features['rate'].rolling(win).max()
    # "roll_mean_{win}" — so'nggi {win} kunlik o'rtacha kurs
    features[f'roll_mean_{win}'] = features['rate'].rolling(win).mean()
    # "roll_median_{win}" — so'nggi {win} kunlik median kurs
    features[f'roll_median_{win}'] = features['rate'].rolling(win).median()
    # "roll_std_{win}" — so'nggi {win} kunlik dispersiya (standart chetlanish)
    #features[f'roll_std_{win}'] = features['rate'].rolling(win).std()

#features.head(35)
# Modelga tayyor bo'lishi uchun NaNlarni tashlab yuboramiz (lag/rolling tufayli paydo bo'lgan)
features_clean = features.dropna().reset_index(drop=True)

# Natijaviy ustunlar ro'yxati
features_clean.head()

,date,rate,lag_1,lag_7,lag_30,roll_min_1,roll_max_1,roll_mean_1,roll_median_1,roll_min_7,roll_max_7,roll_mean_7,roll_median_7,roll_min_30,roll_max_30,roll_mean_30,roll_median_30
0,2018-11-25,"8,277.53","8,277.53","8,271.27","8,226.10","8,277.53","8,277.53","8,277.53","8,277.53","8,271.27","8,277.53","8,276.64","8,277.53","8,226.10","8,277.53","8,255.01","8,249.97"
1,2018-11-26,"8,277.53","8,277.53","8,271.27","8,226.10","8,277.53","8,277.53","8,277.53","8,277.53","8,277.53","8,277.53","8,277.53","8,277.53","8,226.10","8,277.53","8,256.72","8,249.97"
2,2018-11-27,"8,299.20","8,277.53","8,277.53","8,226.10","8,299.20","8,299.20","8,299.20","8,299.20","8,277.53","8,299.20","8,280.63","8,277.53","8,226.10","8,299.20","8,259.16","8,260.62"
3,2018-11-28,"8,299.20","8,299.20","8,277.53","8,226.10","8,299.20","8,299.20","8,299.20","8,299.20","8,277.53","8,299.20","8,283.72","8,277.53","8,236.87","8,299.20","8,261.60","8,271.27"
4,2018-11-29,"8,299.20","8,299.20","8,277.53","8,236.87","8,299.20","8,299.20","8,299.20","8,299.20","8,277.53","8,299.20","8,286.82","8,277.53","8,236.87","8,299.20","8,263.67","8,271.27"


In [35]:
# Kalendar xususiyatlari: kun/oy/chorak va dam olish flaglari (hammasi raqam)
calendar_features = features.copy()
calendar_features['day_of_week'] = calendar_features['date'].dt.dayofweek  # haftaning kuni (0=dushanba)
calendar_features['day_of_month'] = calendar_features['date'].dt.day       # oy kuni (1-31)
calendar_features['month'] = calendar_features['date'].dt.month            # oy (1-12)
calendar_features['quarter'] = calendar_features['date'].dt.quarter        # chorak (1-4)
calendar_features['is_month_start'] = calendar_features['date'].dt.is_month_start.astype(int)  # oy boshimi (1/0)
calendar_features['is_month_end'] = calendar_features['date'].dt.is_month_end.astype(int)      # oy oxirimi (1/0)
calendar_features['is_weekend'] = calendar_features['date'].dt.dayofweek.isin([5,6]).astype(int)  # dam olishmi (1/0)

# Lag/rolling + kalendar ustunlarini tozalaymiz (NaNlarni olib tashlaymiz)
calendar_features_clean = calendar_features.dropna().reset_index(drop=True)
calendar_features_clean.head()
#calendar_features.head(35)


,date,rate,lag_1,lag_7,lag_30,roll_min_1,roll_max_1,roll_mean_1,roll_median_1,roll_min_7,...,roll_max_30,roll_mean_30,roll_median_30,day_of_week,day_of_month,month,quarter,is_month_start,is_month_end,is_weekend
0,2018-11-25,"8,277.53","8,277.53","8,271.27","8,226.10","8,277.53","8,277.53","8,277.53","8,277.53","8,271.27",...,"8,277.53","8,255.01","8,249.97",6,25,11,4,0,0,1
1,2018-11-26,"8,277.53","8,277.53","8,271.27","8,226.10","8,277.53","8,277.53","8,277.53","8,277.53","8,277.53",...,"8,277.53","8,256.72","8,249.97",0,26,11,4,0,0,0
2,2018-11-27,"8,299.20","8,277.53","8,277.53","8,226.10","8,299.20","8,299.20","8,299.20","8,299.20","8,277.53",...,"8,299.20","8,259.16","8,260.62",1,27,11,4,0,0,0
3,2018-11-28,"8,299.20","8,299.20","8,277.53","8,226.10","8,299.20","8,299.20","8,299.20","8,299.20","8,277.53",...,"8,299.20","8,261.60","8,271.27",2,28,11,4,0,0,0
4,2018-11-29,"8,299.20","8,299.20","8,277.53","8,236.87","8,299.20","8,299.20","8,299.20","8,299.20","8,277.53",...,"8,299.20","8,263.67","8,271.27",3,29,11,4,0,0,0
